# Document Augmentation

## 一、理论介绍

Document Augmentation技术利用额外的问题生成来改进矢量数据库中的文档检索。通过生成和合并与每个文本片段相关的各种问题，该系统增强了标准检索过程，从而提高了找到可用作生成式问答上下文的相关文档的可能性。

通过使用相关问题丰富文本片段，我们的目标是显著提高识别文档中包含用户查询答案的最相关部分的准确性。

实现步骤
1. PDF 处理和文本分块：处理 PDF 文档并将其划分为可管理的文本片段。
2. 问题增强：使用 OpenAI 的语言模型在文档和片段级别生成相关问题。
3. 矢量存储创建：使用嵌入模型计算文档的嵌入并创建向量存储。
4. 检索和答案生成：查找最相关的文档，并根据提供的上下文生成答案。

方法优点
1. 增强检索过程：增加为给定查询找到最相关的文档的概率。
2. 灵活的上下文调整：允许轻松调整文本文档和片段的上下文窗口大小。
3. 高质量语言理解：利用 OpenAI 强大的语言模型进行问题生成和答案生成。

## 二、代码实现

### 2.1 一个简单例子

In [26]:
import os
import re
import json

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.vectorstores.chroma import Chroma

from dotenv import load_dotenv
from enum import Enum
from langchain.schema import Document

import warnings
warnings.filterwarnings("ignore")


In [32]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

class QuestionGeneration(Enum):
    """
    枚举类，用于指定文档处理中的问题生成级别。

    属性：
        DOCUMENT_LEVEL (int): 表示在整个文档级别生成问题。
        FRAGMENT_LEVEL (int): 表示在单个文本片段级别生成问题。
    """
    DOCUMENT_LEVEL = 1
    FRAGMENT_LEVEL = 2

# 根据模型的不同，Mitral 7B 的最大值为 8000，Llama 3.1 8B 为 128k
DOCUMENT_MAX_TOKENS = 2000
DOCUMENT_OVERLAP_TOKENS = 400

# 嵌入和文本相似度计算基于较短的文本
FRAGMENT_MAX_TOKENS = 128
FRAGMENT_OVERLAP_TOKENS = 16

# 在文档或片段级别生成的问题
QUESTION_GENERATION = QuestionGeneration.DOCUMENT_LEVEL
# 针对特定文档或片段将生成的问题数量
QUESTIONS_PER_DOCUMENT = 10

In [33]:
class QuestionList(BaseModel):
    question_list: List[str] = Field(..., title="为文档或片段生成的问题列表")


class OpenAIEmbeddingsWrapper(OpenAIEmbeddings):
    """
    OpenAI嵌入的包装类，提供与原始OllamaEmbeddings类似的接口。
    """
    
    def __call__(self, query: str) -> List[float]:
        """
        允许实例作为可调用对象生成查询的嵌入。

        参数：
            query (str): 要嵌入的查询字符串。

        返回：
            List[float]: 查询的嵌入，作为浮点数列表。
        """
        return self.embed_query(query)

def clean_and_filter_questions(questions: List[str]) -> List[str]:
    """
    清理和过滤问题列表。

    参数：
        questions (List[str]): 要清理和过滤的问题列表。

    返回：
        List[str]: 一份清理和过滤后的、以问号结尾的问题列表。
    """
    cleaned_questions = []
    for question in questions:
        cleaned_question = re.sub(r'^\d+\.\s*', '', question.strip())
        if cleaned_question.endswith('？'):
            cleaned_questions.append(cleaned_question)
    return cleaned_questions

def generate_questions(text: str) -> List[str]:
    """
    基于提供的文本生成问题列表，使用OpenAI。

    参数：
        text (str): 用于生成问题的上下文数据。

    返回：
        List[str]: 一份独特的、过滤后的问题列表。
    """
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    prompt = PromptTemplate(
        input_variables=["context", "num_questions"],
        template="使用上下文数据: {context}\n\n生成至少 {num_questions} 个可能的问题，这些问题可以基于该上下文进行提问。确保问题在上下文中是可以直接回答的，并且不包含任何答案或标题。"
                 "用换行符分隔问题。"
    )
    chain = prompt | llm.with_structured_output(QuestionList)
    input_data = {"context": text, "num_questions": QUESTIONS_PER_DOCUMENT}
    result = chain.invoke(input_data)
    
    # 从QuestionList对象中提取问题列表
    questions = result.question_list
    
    filtered_questions = clean_and_filter_questions(questions)
    return list(set(filtered_questions))

def generate_answer(content: str, question: str) -> str:
    """
    基于提供的上下文生成给定问题的答案，使用OpenAI。

    参数：
        content (str): 用于生成答案的上下文数据。
        question (str): 要生成答案的问题。

    返回：
        str: 基于提供的上下文的精确答案。
    """
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="使用上下文数据: {context}\n\n提供对问题: {question} 的简短而精确的答案。"
    )
    chain = prompt | llm
    input_data = {"context": content, "question": question}
    return chain.invoke(input_data)

def split_document(document: str, chunk_size: int, chunk_overlap: int) -> List[str]:
    """
    将文档拆分为较小的文本块。

    参数：
        document (str): 要拆分的文档文本。
        chunk_size (int): 每个块的大小（以标记数量为单位）。
        chunk_overlap (int): 连续块之间的重叠标记数量。

    返回：
        List[str]: 文本块的列表，每个块都是文档内容的字符串。
    """
    tokens = re.findall(r'\b\w+\b', document)
    chunks = []
    for i in range(0, len(tokens), chunk_size - chunk_overlap):
        chunk_tokens = tokens[i:i + chunk_size]
        chunks.append(chunk_tokens)
        if i + chunk_size >= len(tokens):
            break
    return [" ".join(chunk) for chunk in chunks]

def print_document(comment: str, document: str) -> None:
    """
    打印评论，后跟文档的内容。

    参数：
        comment (str): 在文档详细信息之前打印的评论或描述。
        document (str): 要打印其内容的文档。

    返回：
        None
    """
    print(f'{comment} (类型: {document.metadata["type"]}, 索引: {document.metadata["index"]}): {document.page_content}')

In [34]:
# 初始化 OpenAIEmbeddings
embeddings = OpenAIEmbeddingsWrapper()

# 示例文档
example_text = '“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读者通过西瓜书对机器学习有所了解， 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推导细节的读者'

# 生成问题
questions = generate_questions(example_text)
print("生成的问题:")
for q in questions:
    print(f"- {q}")

# 生成答案
sample_question = questions[0] if questions else "这个文档是关于什么的？"
answer = generate_answer(example_text, sample_question)
print(f"\n问题: {sample_question}")
print(f"答案: {answer}")

# 拆分文档
chunks = split_document(example_text, chunk_size=10, chunk_overlap=2)
print("\n文档片段:")
for i, chunk in enumerate(chunks):
    print(f"片段 {i + 1}: {chunk}")

# 使用 OpenAIEmbeddings 的示例
doc_embedding = embeddings.embed_documents([example_text])
query_embedding = embeddings.embed_query("主要主题是什么？")
print("\n文档嵌入（前 5 个元素）:", doc_embedding[0][:5])
print("查询嵌入（前 5 个元素）:", query_embedding[:5])

生成的问题:
- 周志华老师的《机器学习》被称为什么？
- 《机器学习》被认为是经典的什么类型的教材？
- 西瓜书的全名是什么？
- 《机器学习》是哪个领域的教材？
- 《机器学习》是否适合想要深入研究机器学习的读者？
- 周志华老师在书中对公式推导的细节是如何处理的？
- 《机器学习》这本书的作者是谁？
- 《机器学习》这本书的出版目的是什么？
- 为什么周志华老师没有详述部分公式的推导细节？
- 《机器学习》这本书的目标读者是谁？

问题: 周志华老师的《机器学习》被称为什么？
答案: content='周志华老师的《机器学习》被称为“西瓜书”。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 115, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None} id='run-0328dfac-fb8a-496f-b94f-fe019e99b8f4-0' usage_metadata={'input_tokens': 115, 'output_tokens': 18, 'total_tokens': 133, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

文档片段:
片段 1: 周志华老师

### 2.2 具体实现

In [35]:
def clean_text(text: str):
    """
    Implement text cleaning function

    Args:
        text: Field requiring cleaning

    Returns:
        Field returned after cleaning is completed
    
    """
    # 删除每页开头与结尾标语及链接
    text = re.sub(r'→_→\n欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解》\n←_←', '', text)
    text = re.sub(r'→_→\n配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU\n←_←', '', text)
    # 删除字符串开头的空格
    text = re.sub(r'\s+', '', text)
    # 删除回车
    text = re.sub(r'\n+', '', text)

    return text

def DA_retriever(content: str, embedding_model: OpenAIEmbeddings):
    """
    处理文档内容，将其拆分为片段，生成问题，
    创建向量存储，并返回检索器。

    参数：
        content (str): 要处理的文档内容。
        embedding_model (OpenAIEmbeddings): 用于向量化的嵌入模型。

    返回：
        VectorStoreRetriever: 用于检索最相关的文档的检索器。
    """
    # 将整个文本内容拆分为文本文档
    text_documents = split_document(content, DOCUMENT_MAX_TOKENS, DOCUMENT_OVERLAP_TOKENS)
    print(f'文本内容拆分为: {len(text_documents)} 个文档')

    documents = []
    counter = 0
    for i, text_document in enumerate(text_documents):
        text_fragments = split_document(text_document, FRAGMENT_MAX_TOKENS, FRAGMENT_OVERLAP_TOKENS)
        print(f'文本文档 {i} - 拆分为: {len(text_fragments)} 个片段')
        
        for j, text_fragment in enumerate(text_fragments):
            documents.append(Document(
                page_content=text_fragment,
                metadata={"type": "ORIGINAL", "index": counter, "text": text_document}
            ))
            counter += 1
            
            if QUESTION_GENERATION == QuestionGeneration.FRAGMENT_LEVEL:
                questions = generate_questions(text_fragment)
                documents.extend([
                    Document(page_content=question, metadata={"type": "AUGMENTED", "index": counter + idx, "text": text_document})
                    for idx, question in enumerate(questions)
                ])
                counter += len(questions)
                print(f'文本文档 {i} 文本片段 {j} - 生成: {len(questions)} 个问题')
        
        if QUESTION_GENERATION == QuestionGeneration.DOCUMENT_LEVEL:
            questions = generate_questions(text_document)
            documents.extend([
                Document(page_content=question, metadata={"type": "AUGMENTED", "index": counter + idx, "text": text_document})
                for idx, question in enumerate(questions)
            ])
            counter += len(questions)
            print(f'文本文档 {i} - 生成: {len(questions)} 个问题')

    for document in documents:
        print_document("数据集", document)

    print(f'创建存储，计算 {len(documents)} 个 文档的嵌入')
    vectorstore = Chroma.from_documents(documents, embedding_model)

    print("创建检索器以返回最相关的 Chroma 文档")
    return vectorstore.as_retriever(search_kwargs={"k": 3})

In [36]:
pdf_path = "data/pumpkin_book.pdf"
qa_path = 'data/train_dataset.json'
embedding = HuggingFaceEmbeddings(model_name='BAAI/bge-small-zh-v1.5')

with open(qa_path, 'r', encoding='utf-8') as file:
    qa_pairs = json.load(file)

In [37]:
# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径，加载PDF
loader = PyMuPDFLoader(pdf_path)

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pdf_pages = loader.load()

# 第13页为南瓜书第一页正文，因此从13页开始,从倒数13页涉及敏感用语，因此从-13页结束
data_pages = pdf_pages[13:-13]

为了减小token消耗，这里只处理page_num = 13的PDF数据，同时基于已经处理好的qa_pairs中的第一个进行试验

In [38]:
qa_pairs[0]

{'query': '请根据提供的上下文信息，解释“算法”和“模型”的概念，并说明它们在机器学习中的关系。',
 'answer': '“算法”是指从数据中学得“模型”的具体方法，例如后续章节中将会讲述的线性回归、对数几率回归、决策树等。“算法”产出的结果称为“模型”，通常是具体的函数或者可抽象地看作为函数，例如一元线性回归算法产出的模型即为形如f(x)=wx+b的一元一次函数。不过由于严格区分这两者的意义不大，因此多数文献和资料会将其混用，当遇到这两个概念时，其具体指代根据上下文判断即可。',
 'page_num': 13}

In [39]:
# 只使用page_num = 13
data_pages_sample = data_pages[0]

content = clean_text(data_pages_sample.page_content)
content

'第1章绪论本章作为“西瓜书”的开篇，主要讲解什么是机器学习以及机器学习的相关数学符号，为后续内容作铺垫，并未涉及复杂的算法理论，因此阅读本章时只需耐心梳理清楚所有概念和数学符号即可。此外，在阅读本章前建议先阅读西瓜书目录前页的《主要符号表》，它能解答在阅读“西瓜书”过程中产生的大部分对数学符号的疑惑。本章也作为本书的开篇，笔者在此赘述一下本书的撰写初衷，本书旨在以“过来人”的视角陪读者一起阅读“西瓜书”，尽力帮读者消除阅读过程中的“数学恐惧”，只要读者学习过《高等数学》、《线性代数》和《概率论与数理统计》这三门大学必修的数学课，均能看懂本书对西瓜书中的公式所做的解释和推导，同时也能体会到这三门数学课在机器学习上碰撞产生的“数学之美”。1.1引言本节以概念理解为主，在此对“算法”和“模型”作补充说明。“算法”是指从数据中学得“模型”的具体方法，例如后续章节中将会讲述的线性回归、对数几率回归、决策树等。“算法”产出的结果称为“模型”，通常是具体的函数或者可抽象地看作为函数，例如一元线性回归算法产出的模型即为形如f(x)=wx+b的一元一次函数。不过由于严格区分这两者的意义不大，因此多数文献和资料会将其混用，当遇到这两个概念时，其具体指代根据上下文判断即可。1.2基本术语本节涉及的术语较多且很多术语都有多个称呼，下面梳理各个术语，并将最常用的称呼加粗标注。样本：也称为“示例”，是关于一个事件或对象的描述。因为要想让计算机能对现实生活中的事物进行机器学习，必须先将其抽象为计算机能理解的形式，计算机最擅长做的就是进行数学运算，因此考虑如何将其抽象为某种数学形式。显然，线性代数中的向量就很适合，因为任何事物都可以由若干“特征”（或称为“属性”）唯一刻画出来，而向量的各个维度即可用来描述各个特征。例如，如果用色泽、根蒂和敲声这3个特征来刻画西瓜，那么一个“色泽青绿，根蒂蜷缩，敲声清脆”的西瓜用向量来表示即为x=(青绿;蜷缩;清脆)（向量中的元素用分号“;”分隔时表示此向量为列向量，用逗号“,”分隔时表示为行向量），其中青绿、蜷缩和清脆分别对应为相应特征的取值，也称为“属性值”。显然，用中文书写向量的方式不够“数学”，因此需要将属性值进一步数值化，具体例子参见“西瓜书”第3章3.2。此外，仅靠以上3个特征来刻画西瓜显然不够全面细致，因此还需要扩展更多维度的特征，一般称此类与特征处

In [40]:
# 处理文档并创建检索器
document_query_retriever = DA_retriever(content, embedding)

文本内容拆分为: 1 个文档
文本文档 0 - 拆分为: 2 个片段
文本文档 0 - 生成: 10 个问题
数据集 (类型: ORIGINAL, 索引: 0): 第1章绪论本章作为 西瓜书 的开篇 主要讲解什么是机器学习以及机器学习的相关数学符号 为后续内容作铺垫 并未涉及复杂的算法理论 因此阅读本章时只需耐心梳理清楚所有概念和数学符号即可 此外 在阅读本章前建议先阅读西瓜书目录前页的 主要符号表 它能解答在阅读 西瓜书 过程中产生的大部分对数学符号的疑惑 本章也作为本书的开篇 笔者在此赘述一下本书的撰写初衷 本书旨在以 过来人 的视角陪读者一起阅读 西瓜书 尽力帮读者消除阅读过程中的 数学恐惧 只要读者学习过 高等数学 线性代数 和 概率论与数理统计 这三门大学必修的数学课 均能看懂本书对西瓜书中的公式所做的解释和推导 同时也能体会到这三门数学课在机器学习上碰撞产生的 数学之美 1 1引言本节以概念理解为主 在此对 算法 和 模型 作补充说明 算法 是指从数据中学得 模型 的具体方法 例如后续章节中将会讲述的线性回归 对数几率回归 决策树等 算法 产出的结果称为 模型 通常是具体的函数或者可抽象地看作为函数 例如一元线性回归算法产出的模型即为形如f x wx b的一元一次函数 不过由于严格区分这两者的意义不大 因此多数文献和资料会将其混用 当遇到这两个概念时 其具体指代根据上下文判断即可 1 2基本术语本节涉及的术语较多且很多术语都有多个称呼 下面梳理各个术语 并将最常用的称呼加粗标注 样本 也称为 示例 是关于一个事件或对象的描述 因为要想让计算机能对现实生活中的事物进行机器学习 必须先将其抽象为计算机能理解的形式 计算机最擅长做的就是进行数学运算 因此考虑如何将其抽象为某种数学形式 显然 线性代数中的向量就很适合 因为任何事物都可以由若干 特征 或称为 属性 唯一刻画出来 而向量的各个维度即可用来描述各个特征 例如 如果用色泽 根蒂和敲声这3个特征来刻画西瓜 那么一个 色泽青绿 根蒂蜷缩 敲声清脆 的西瓜用向量来表示即为x 青绿 蜷缩 清脆 向量中的元素用分号 分隔时表示此向量为列向量 用逗号 分隔时表示为行向量 其中青绿 蜷缩和清脆分别对应为相应特征的取值 也称为 属性值 显然 用中文书写向量的方式不够 数学 因此需要将属性值进一步数值化 具体例子参见 西

In [41]:
query = qa_pairs[0]['query']
retrieved_docs = document_query_retriever.get_relevant_documents(query)
print(f"\nQuery: {query}")
print(f"Retrieved document: {retrieved_docs[0].page_content}")
print(f"Retrieved document original text: {retrieved_docs[0].metadata['text']}")


Query: 请根据提供的上下文信息，解释“算法”和“模型”的概念，并说明它们在机器学习中的关系。
Retrieved document: 算法和模型之间的区别是什么？
Retrieved document original text: 第1章绪论本章作为 西瓜书 的开篇 主要讲解什么是机器学习以及机器学习的相关数学符号 为后续内容作铺垫 并未涉及复杂的算法理论 因此阅读本章时只需耐心梳理清楚所有概念和数学符号即可 此外 在阅读本章前建议先阅读西瓜书目录前页的 主要符号表 它能解答在阅读 西瓜书 过程中产生的大部分对数学符号的疑惑 本章也作为本书的开篇 笔者在此赘述一下本书的撰写初衷 本书旨在以 过来人 的视角陪读者一起阅读 西瓜书 尽力帮读者消除阅读过程中的 数学恐惧 只要读者学习过 高等数学 线性代数 和 概率论与数理统计 这三门大学必修的数学课 均能看懂本书对西瓜书中的公式所做的解释和推导 同时也能体会到这三门数学课在机器学习上碰撞产生的 数学之美 1 1引言本节以概念理解为主 在此对 算法 和 模型 作补充说明 算法 是指从数据中学得 模型 的具体方法 例如后续章节中将会讲述的线性回归 对数几率回归 决策树等 算法 产出的结果称为 模型 通常是具体的函数或者可抽象地看作为函数 例如一元线性回归算法产出的模型即为形如f x wx b的一元一次函数 不过由于严格区分这两者的意义不大 因此多数文献和资料会将其混用 当遇到这两个概念时 其具体指代根据上下文判断即可 1 2基本术语本节涉及的术语较多且很多术语都有多个称呼 下面梳理各个术语 并将最常用的称呼加粗标注 样本 也称为 示例 是关于一个事件或对象的描述 因为要想让计算机能对现实生活中的事物进行机器学习 必须先将其抽象为计算机能理解的形式 计算机最擅长做的就是进行数学运算 因此考虑如何将其抽象为某种数学形式 显然 线性代数中的向量就很适合 因为任何事物都可以由若干 特征 或称为 属性 唯一刻画出来 而向量的各个维度即可用来描述各个特征 例如 如果用色泽 根蒂和敲声这3个特征来刻画西瓜 那么一个 色泽青绿 根蒂蜷缩 敲声清脆 的西瓜用向量来表示即为x 青绿 蜷缩 清脆 向量中的元素用分号 分隔时表示此向量为列向量 用逗号 分隔时表示为行向量 其中青绿 蜷缩和清脆分别对应为相应特征的取值 也称为 属性值 显然 用